# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from plotly.subplots import make_subplots
from pytorch_tabular import TabularModel
import torch
import plotly.graph_objects as go
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.impute import KNNImputer
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import datetime
from collections import Counter
from matplotlib.ticker import MaxNLocator
from itertools import chain
from sklearn.metrics import mean_absolute_error
import pyaging as pya
import matplotlib.lines as mlines
from src.models.simage.tabular.widedeep.ft_transformer import WDFTTransformerModel
import statsmodels.formula.api as smf
from itertools import chain
from pingouin import ancova
from sklearn.preprocessing import LabelEncoder 
import upsetplot


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]


def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


def get_sections(sets):
    """
    Given a list of sets, return a new list of sets with all the possible
    mutually exclusive overlapping combinations of those sets.  Another way
    to think of this is the mutually exclusive sections of a venn diagram
    of the sets.  If the original list has N sets, the returned list will
    have (2**N)-1 sets.

    Parameters
    ----------
    sets : list of set

    Returns
    -------
    combinations : list of tuple
        tag : str
            Binary string representing which sets are included / excluded in
            the combination.
        set : set
            The set formed by the overlapping input sets.
    """
    num_combinations = 2 ** len(sets)
    bit_flags = [2 ** n for n in range(len(sets))]
    flags_zip_sets = [z for z in zip(bit_flags, sets)]

    combo_sets = {}
    for bits in range(num_combinations - 1, 0, -1):
        include_sets = [s for flag, s in flags_zip_sets if bits & flag]
        exclude_sets = [s for flag, s in flags_zip_sets if not bits & flag]
        combo = set.intersection(*include_sets)
        combo = set.difference(combo, *exclude_sets)
        tag = ''.join([str(int((bits & flag) > 0)) for flag in bit_flags])
        combo_sets[tag] = combo
    return combo_sets


# Select 20 samples

In [ ]:
path_load = f"E:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno"

df = pd.read_excel(f"{path_load}/data.xlsx", index_col=0)
df = df.loc[df['Region'] == 'Mirny', :]

path_save = f"E:/YandexDisk/Work/bbd/mirny/select_20_samples"

ids_groups = {
    'All': df.index.values,
}
colors_groups = {
    'All': 'crimson',
    'Selected': 'dodgerblue',
}

ids_groups['Selected'] = [
    'M90',
    'M106',
    'M205',
    'M230',
    'M105',
    'M33',
    'M189',
    'M214',
    'M226',
    'M201',
    'M196',
    'M1',
    'M89',
    'M163',
    'M25',
    'M35',
    'M241',
    'M215',
    'M210',
    'M170'
]

# Plot age histogram
df_fig_1 = df.loc[ids_groups['All'], ['Age']].copy()
df_fig_1['Group'] = 'All'
df_fig_2 = df.loc[ids_groups['Selected'], ['Age']].copy()
df_fig_2['Group'] = 'Selected'
df_fig = pd.concat([df_fig_1, df_fig_2], ignore_index=True)
hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='whitegrid')
fig, ax = plt.subplots(figsize=(6, 4))
histplot = sns.histplot(
    data=df_fig,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Age",
    hue='Group',
    palette=colors_groups,
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path_save}/hist_age.png", bbox_inches='tight', dpi=200)
# plt.savefig(f"{path_save}/hist_age.pdf", bbox_inches='tight')
plt.close(fig)

df.loc[ids_groups['Selected'], ['Age', 'Sex']].to_excel(f"{path_save}/selected.xlsx")

## Select additional 20 samples

In [ ]:
path_load = f"E:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno"

df = pd.read_excel(f"{path_load}/data.xlsx", index_col=0)
df = df.loc[df['Region'] == 'Mirny', :]

path_save = f"E:/YandexDisk/Work/bbd/mirny/select_20_samples"

df_already_selected = pd.read_excel(f"{path_save}/selected.xlsx", index_col=0)

df = df.loc[df.index.difference(df_already_selected.index), :]

In [ ]:

ids_groups = {
    'All': df.index.values,
}
colors_groups = {
    'All': 'crimson',
    'Selected': 'dodgerblue',
}

ids_groups['Selected'] = [
    'M211',
    'M202',
    'M188',
    'M3',
    'M216',
    'M217',
    'M234',
    'M183',
    'M229',
    'M233',
    'M18',
    'M113',
    'M108',
    'M213',
    'M110',
    'M96',
    'M111',
    'M104',
    'M83',
    'M101',
]

# Plot age histogram
df_fig_1 = df.loc[ids_groups['All'], ['Age']].copy()
df_fig_1['Group'] = 'All'
df_fig_2 = df.loc[ids_groups['Selected'], ['Age']].copy()
df_fig_2['Group'] = 'Selected'
df_fig = pd.concat([df_fig_1, df_fig_2], ignore_index=True)
hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='whitegrid')
fig, ax = plt.subplots(figsize=(6, 4))
histplot = sns.histplot(
    data=df_fig,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Age",
    hue='Group',
    palette=colors_groups,
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path_save}/hist_age_new_20.png", bbox_inches='tight', dpi=200)
plt.close(fig)

df.loc[ids_groups['Selected'], ['Age', 'Sex']].to_excel(f"{path_save}/selected_new_20.xlsx")